# Выбор метрики для детектирования линий на основе человеческой разметки

In [1]:
import pandas as pd
from collections import defaultdict
from sklearn.metrics import f1_score
from imblearn.over_sampling import RandomOverSampler

In [2]:
bdd_df = pd.read_csv("BDD10k_metric_stats.csv", index_col="img_id")
cityscapes_df = pd.read_csv("CityScapes_metric_stats.csv", index_col="img_id")
cityscapes_rain_df = pd.read_csv("CityScapesRain_metric_stats.csv", index_col="img_id")

In [3]:
metrics = list(bdd_df.columns)
metrics.remove("human")

In [53]:
# Удаляет все дубликаты и ответы "Затрудняюсь ответить"
# Перераспределяет результаты чтобы избежать несбалансированности
def prepare_df(df):
    df = df.copy()
    df = df[df["human"] != 0.5]
    df = df[~df.index.duplicated(keep='first')]
    ros = RandomOverSampler(random_state=0)
    X, y = ros.fit_resample(df[metrics], df["human"])
    X["human"] = y
    return X

# Считает F1
def get_score(df):
    md = []
    for m in metrics:
        md.append((m, df[m].sub(df["human"]).abs().mean(axis=0)))
    md = sorted(md, key=lambda x: x[1])
    return md

In [6]:
bdd_df = prepare_df(bdd_df)
cityscapes_df = prepare_df(cityscapes_df)
cityscapes_rain_df = prepare_df(cityscapes_rain_df)

In [7]:
bdd_df["human"].mean()

0.5

In [8]:
cityscapes_df["human"].mean()

0.5

In [9]:
cityscapes_rain_df["human"].mean()

0.5

## Оптимальная метрика для каждого датасета

In [54]:
get_score(bdd_df)

[('NFOM_fp_0.2_fn_0.2_reg_0', 0.30833333333333335),
 ('NFOM_fp_0.1_fn_0.2_reg_0.5', 0.31666666666666665),
 ('Jaccard_reg_0.2', 0.325),
 ('NFOM_fp_0.2_fn_0.1_reg_0', 0.3333333333333333),
 ('NFOM_fp_0.2_fn_0.2_reg_0.2', 0.3333333333333333),
 ('Dice_reg_0.5', 0.3333333333333333),
 ('NFOM_fp_0.2_fn_0.2_reg_0.5', 0.3333333333333333),
 ('Dice_reg_0.2', 0.3416666666666667),
 ('NFOM_fp_0.1_fn_0.4_reg_0.5', 0.3416666666666667),
 ('NFOM_fp_0.1_fn_0.2_reg_0', 0.35),
 ('NFOM_fp_0.2_fn_0.1_reg_0.2', 0.35),
 ('Dice_reg_0', 0.36666666666666664),
 ('Jaccard_reg_0', 0.36666666666666664),
 ('Jaccard_reg_0.5', 0.36666666666666664),
 ('NFOM_fp_0.2_fn_0.1_reg_0.5', 0.36666666666666664),
 ('NFOM_fp_0.4_fn_0.1_reg_0.5', 0.36666666666666664),
 ('NFOM_fp_0.1_fn_0.4_reg_0.2', 0.375),
 ('NFOM_fp_0.1_fn_0.2_reg_0.2', 0.375),
 ('NFOM_fp_0.1_fn_0.4_reg_0', 0.38333333333333336),
 ('NFOM_fp_0.4_fn_0.1_reg_0.2', 0.38333333333333336),
 ('NFOM_fp_0.4_fn_0.1_reg_0', 0.4),
 ('SSIM_reg_0', 0.49166666666666664),
 ('MSE_reg_

In [30]:
get_score(cityscapes_df)

[('Jaccard_reg_0.5', 0.3765432098765432),
 ('NFOM_fp_0.4_fn_0.1_reg_0', 0.38271604938271603),
 ('NFOM_fp_0.2_fn_0.1_reg_0.5', 0.3888888888888889),
 ('NFOM_fp_0.2_fn_0.1_reg_0.2', 0.4012345679012346),
 ('NFOM_fp_0.4_fn_0.1_reg_0.2', 0.41358024691358025),
 ('SSIM_reg_0', 0.41975308641975306),
 ('NFOM_fp_0.4_fn_0.1_reg_0.5', 0.43209876543209874),
 ('Dice_reg_0.5', 0.4444444444444444),
 ('NFOM_fp_0.2_fn_0.2_reg_0.2', 0.4506172839506173),
 ('NFOM_fp_0.1_fn_0.2_reg_0.5', 0.4506172839506173),
 ('NFOM_fp_0.2_fn_0.2_reg_0', 0.4567901234567901),
 ('NFOM_fp_0.2_fn_0.1_reg_0', 0.4567901234567901),
 ('NFOM_fp_0.2_fn_0.2_reg_0.5', 0.4567901234567901),
 ('Dice_reg_0.2', 0.4691358024691358),
 ('Jaccard_reg_0.2', 0.4691358024691358),
 ('NFOM_fp_0.1_fn_0.2_reg_0.2', 0.4691358024691358),
 ('MSE_reg_0', 0.48148148148148145),
 ('PSNR_reg_0', 0.48148148148148145),
 ('PSNR_reg_0.2', 0.48148148148148145),
 ('NFOM_fp_0.1_fn_0.4_reg_0', 0.4876543209876543),
 ('NFOM_fp_0.1_fn_0.2_reg_0', 0.4876543209876543),
 ('

In [31]:
get_score(cityscapes_rain_df)

[('SSIM_reg_0', 0.46703296703296704),
 ('MSE_reg_0', 0.4945054945054945),
 ('PSNR_reg_0', 0.4945054945054945),
 ('PSNR_reg_0.2', 0.4945054945054945),
 ('PSNR_reg_0.5', 0.4945054945054945),
 ('NFOM_fp_0.1_fn_0.4_reg_0', 0.5),
 ('NFOM_fp_0.1_fn_0.2_reg_0', 0.5),
 ('NFOM_fp_0.2_fn_0.2_reg_0', 0.5),
 ('MSE_reg_0.2', 0.5),
 ('SSIM_reg_0.2', 0.5),
 ('Jaccard_reg_0.2', 0.5),
 ('NFOM_fp_0.1_fn_0.4_reg_0.2', 0.5),
 ('NFOM_fp_0.1_fn_0.2_reg_0.2', 0.5),
 ('MSE_reg_0.5', 0.5),
 ('SSIM_reg_0.5', 0.5),
 ('NFOM_fp_0.1_fn_0.4_reg_0.5', 0.5),
 ('NFOM_fp_0.1_fn_0.2_reg_0.5', 0.5),
 ('Dice_reg_0', 0.5054945054945055),
 ('Jaccard_reg_0', 0.5054945054945055),
 ('Dice_reg_0.2', 0.5054945054945055),
 ('NFOM_fp_0.2_fn_0.2_reg_0.2', 0.5054945054945055),
 ('Dice_reg_0.5', 0.5054945054945055),
 ('NFOM_fp_0.2_fn_0.2_reg_0.5', 0.5054945054945055),
 ('NFOM_fp_0.2_fn_0.1_reg_0', 0.510989010989011),
 ('NFOM_fp_0.4_fn_0.1_reg_0', 0.5164835164835165),
 ('NFOM_fp_0.2_fn_0.1_reg_0.2', 0.5164835164835165),
 ('NFOM_fp_0.4_

## Оптимальная метрика для обьединенных данных

In [32]:
bdd_df.reset_index(drop=True, inplace=True)
cityscapes_df.reset_index(drop=True, inplace=True)
cityscapes_rain_df.reset_index(drop=True, inplace=True)
all_data = pd.concat([bdd_df, cityscapes_df, cityscapes_rain_df])
get_score(all_data)

[('Jaccard_reg_0.5', 0.42887931034482757),
 ('NFOM_fp_0.2_fn_0.1_reg_0.2', 0.4331896551724138),
 ('NFOM_fp_0.2_fn_0.1_reg_0.5', 0.4331896551724138),
 ('NFOM_fp_0.2_fn_0.2_reg_0', 0.4353448275862069),
 ('NFOM_fp_0.1_fn_0.2_reg_0.5', 0.4353448275862069),
 ('NFOM_fp_0.4_fn_0.1_reg_0', 0.4396551724137931),
 ('Dice_reg_0.5', 0.4396551724137931),
 ('NFOM_fp_0.2_fn_0.2_reg_0.2', 0.4418103448275862),
 ('Jaccard_reg_0.2', 0.44396551724137934),
 ('NFOM_fp_0.2_fn_0.2_reg_0.5', 0.44396551724137934),
 ('NFOM_fp_0.2_fn_0.1_reg_0', 0.44612068965517243),
 ('NFOM_fp_0.4_fn_0.1_reg_0.2', 0.44612068965517243),
 ('Dice_reg_0.2', 0.4504310344827586),
 ('NFOM_fp_0.4_fn_0.1_reg_0.5', 0.4504310344827586),
 ('NFOM_fp_0.1_fn_0.4_reg_0.5', 0.4547413793103448),
 ('SSIM_reg_0', 0.45689655172413796),
 ('NFOM_fp_0.1_fn_0.2_reg_0', 0.45689655172413796),
 ('NFOM_fp_0.1_fn_0.2_reg_0.2', 0.45689655172413796),
 ('NFOM_fp_0.1_fn_0.4_reg_0.2', 0.46336206896551724),
 ('Dice_reg_0', 0.46551724137931033),
 ('Jaccard_reg_0', 0